<a href="https://colab.research.google.com/github/rsrohan/Energy-Utilization-of-Rooftops-In-Urban-Areas/blob/segmentation/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# @markdown Loading Dependencies
!pip install patool
import patoolib
from distutils.dir_util import copy_tree
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import save_img 

# %tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
patoolib.extract_archive("/content/gdrive/My Drive/nerve.rar", outdir="/content")
src = '/content/nerve/nerve/model'
dst = '/content/model'

copy_tree(src, dst)

In [0]:
data_dir = '/content'
input_size = 256
epochs = 5
batch_size = 16

orig_width = 580
orig_height = 420

threshold = 0.5

In [0]:
from model.u_net import get_unet_256

model = get_unet_256()

In [0]:
df_train = pd.read_csv('/content/gdrive/My Drive/minor/train_masks2.csv')

df_train.head()

,Unnamed: 0,img,pixels
0,0,10066_0,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
1,1,10066_1,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
2,2,10066_2,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
3,3,10066_3,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
4,4,10066_4,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...


In [0]:
# @markdown Selecting number of images per camera for training
for index,rows in df_train.iterrows():
  if int(rows['img'].split('_')[1]) < 120:
    pass
  else:
    indexes.append(index)
df_train=df_train.drop(df_train.index[indexes])

In [0]:
ids_train = df_train['img']
print(ids_train.head())

0    10066_0
1    10066_1
2    10066_2
3    10066_3
4    10066_4
Name: img, dtype: object


In [0]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.1, random_state=42)

print('Training on {} samples'.format(len(ids_train_split)))
print('Validating on {} samples'.format(len(ids_valid_split)))

Training on 4968 samples
Validating on 552 samples


In [0]:
# @markdown Image Processing
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask

def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask

In [0]:
# @markdown Function for training
def train_generator():
    
    while True:
        for start in range(0, len(ids_train_split), batch_size):
            
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_train_split))
            ids_train_batch = ids_train_split[start:end]
            #print(ids_train_batch.values)

            for id in ids_train_batch.values:
                print('train/content/gdrive/My Drive/minor/dataset/{}/{}.png'.format(id.split('_')[0],id))
                img = cv2.imread(os.path.join(data_dir, '/content/gdrive/My Drive/minor/dataset/{}/{}.png'.format(id.split('_')[0],id)))
                #print(img)
                img = cv2.resize(img, (input_size, input_size))    
                mask = cv2.imread(os.path.join(data_dir, '/content/gdrive/My Drive/minor/img_masks/{}.png'.format(id.split('_')[0])), cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (input_size, input_size))
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            
            yield x_batch, y_batch

In [0]:
# @markdown Function for validating

def valid_generator():
    input_size = 256
    while True:
        for start in range(0, len(ids_valid_split), batch_size):
            
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            #print(ids_valid_batch)
            for id in ids_valid_batch.values:
                #print('valid/content/gdrive/My Drive/minor/dataset/{}/{}.png'.format(id.split('_')[0],id))
                img = cv2.imread(os.path.join(data_dir, '/content/gdrive/My Drive/minor/dataset/{}/{}.png'.format(id.split('_')[0],id)))
                img = cv2.resize(img, (input_size, input_size))
                mask = cv2.imread(os.path.join(data_dir, '/content/gdrive/My Drive/minor/img_masks/{}.png'.format(id.split('_')[0])), cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (input_size, input_size))
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            
            yield x_batch, y_batch

In [0]:
# @markdown Function for Model-fitting

callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
             ModelCheckpoint(monitor='val_loss', filepath='/content/gdrive/My Drive/minor/segmentation_weights.h5', save_best_only=False,
                             save_weights_only=True)]

model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))


train/content/gdrive/My Drive/minor/dataset/861/861_105.png
valid/content/gdrive/My Drive/minor/dataset/19834/19834_55.png
Epoch 1/5
valid/content/gdrive/My Drive/minor/dataset/7371/7371_68.png
train/content/gdrive/My Drive/minor/dataset/10917/10917_96.png
valid/content/gdrive/My Drive/minor/dataset/4232/4232_43.png
valid/content/gdrive/My Drive/minor/dataset/9708/9708_90.png


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:718: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


Streaming output truncated to the last 5000 lines.
train/content/gdrive/My Drive/minor/dataset/3395/3395_118.png
train/content/gdrive/My Drive/minor/dataset/18590/18590_89.png
train/content/gdrive/My Drive/minor/dataset/9112/9112_68.png
train/content/gdrive/My Drive/minor/dataset/10870/10870_112.png
train/content/gdrive/My Drive/minor/dataset/10066/10066_66.png
247/311 [======================>.......] - ETA: 8:24 - loss: 0.2472 - dice_coeff: 0.8985train/content/gdrive/My Drive/minor/dataset/19834/19834_43.png
train/content/gdrive/My Drive/minor/dataset/5021/5021_43.png
train/content/gdrive/My Drive/minor/dataset/65/65_116.png
train/content/gdrive/My Drive/minor/dataset/861/861_72.png
train/content/gdrive/My Drive/minor/dataset/10870/10870_5.png
train/content/gdrive/My Drive/minor/dataset/19106/19106_96.png
train/content/gdrive/My Drive/minor/dataset/3297/3297_56.png
train/content/gdrive/My Drive/minor/dataset/9112/9112_69.png
train/content/gdrive/My Drive/minor/dataset/9112/9112_31.png

In [0]:
# @markdown Converting test image into RLE
model.load_weights('/content/gdrive/My Drive/minor/segmentation_weights2.h5')
x_batch=[]

def run_length_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    print(img)
    pixels = img.transpose().flatten()
    print(pixels)
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

img = cv2.imread('/content/fbb5e00c-b05c-437a-84f4-2993c6e187b3.jfif')
img = cv2.resize(img, (input_size, input_size))
x_batch.append(img)
        
x_batch = np.array(x_batch, np.float32) / 255

preds = model.predict(x_batch)
preds = np.squeeze(preds, axis=3)
print(preds)
prob = cv2.resize(preds[0], (orig_width, orig_height))
mask = prob > threshold
print(mask)
rle = run_length_encode(mask)

In [0]:
# @markdown Converting RLE to image
def rle_decode(mask_rle, shape=(orig_width,orig_height)):
    print('rle_decode(mask_rle = ', mask_rle)
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction
imgg= rle_decode(rle)

In [0]:
# @markdown Saving image
f_name = 'test8.jfif'
out_path = os.path.join('/content', f_name)
save_img(out_path, imgg[..., np.newaxis])